# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://elephantventures.com")
ed.links

['https://www.elephantventures.com/data-to-decision/',
 'https://www.elephantventures.com/data-to-decision-data-engineering-2/',
 'https://www.elephantventures.com/data-to-decision-rapid-planning/',
 'https://www.elephantventures.com/data-to-decision-data-visualizations/',
 'https://www.elephantventures.com/enterprise-api/',
 'https://www.elephantventures.com/ai-services/',
 'https://www.elephantventures.com/aws-amazon-bedrock-and-q-poc/',
 'https://www.elephantventures.com/generative-ai/',
 'https://www.elephantventures.com/gen-ai-design-sprint/',
 'https://www.elephantventures.com/blog/',
 '/#contact',
 'https://www.elephantventures.com/aws-amazon-bedrock-and-q-poc/',
 'mailto:info@elephantventures.com']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://elephantventures.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.elephantventures.com/data-to-decision/
https://www.elephantventures.com/data-to-decision-data-engineering-2/
https://www.elephantventures.com/data-to-decision-rapid-planning/
https://www.elephantventures.com/data-to-decision-data-visualizations/
https://www.elephantventures.com/enterprise-api/
https://www.elephantventures.com/ai-services/
https://www.elephantventures.com/aws-amazon-bedrock-and-q-poc/
https://www.elephantventures.com/generative-ai/
https://www.elephantventures.com/gen-ai-design-sprint/
https://www.elephantventures.com/blog/
/#contact
https://www.elephantventures.com/aws-amazon-bedrock-and-q-poc/
mailto:info@elephantventures.com


In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/QwQ-32B',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/Phi-4-multimodal-instruct',
 '/Wan-AI/Wan2.1-T2V-14B',
 '/CohereForAI/aya-vision-8b',
 '/models',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/ASLP-lab/DiffRhythm',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces',
 '/datasets/facebook/natural_reasoning',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/GeneralReasoning/GeneralThought-195K',
 '/datasets/KodCode/KodCode-V1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gramm

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/QwQ-32B
Updated
2 days ago
•
103k
•
1.66k
deepseek-ai/DeepSeek-R1
Updated
13 days ago
•
3.64M
•
11k
microsoft/Phi-4-multimodal-instruct
Updated
1 day ago
•
231k
•
1.03k
Wan-AI/Wan2.1-T2V-14B
Updated
11 days ago
•
186k
•
949
CohereForAI/aya-vision-8b
Updated
5 days ago
•
144k
•
206
Browse 1M+ models
Spaces
Runni

In [30]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short villain-like, snarkey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt # Truncate if more than 5,000 characters
    return user_prompt

In [25]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/QwQ-32B\nUpdated\n2 days ago\n•\n103k\n•\n1.66k\ndeepseek-ai/DeepSeek-R1\nUpdated\n13 days ago\n•\n3.64M\n•\n11k\nmicrosoft/Phi-4-multimodal-instruct\nUpdated\n1 day ago\n•\n231k\n•\n1.03k\nWan-AI/Wan2.1-T2V-14B\nUpdated\n11 days ago\n•\n186k\n•\n949\nCohereForAI/aya-vision-8b\nUpdated\n5 days ago\n•\n144k\n•\n206\nBrowse 1M+ models\nSpaces\nRunning\n1.03k\n1.03k\nWan2.1\n💻\nWan: Open and

In [26]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [27]:
create_brochure("Elephant Ventures, LLC.", "https://elephantventures.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.elephantventures.com/data-to-decision/'}, {'type': 'services page', 'url': 'https://www.elephantventures.com/ai-services/'}, {'type': 'blog page', 'url': 'https://www.elephantventures.com/blog/'}]}


```markdown
# Elephant Ventures, LLC: Digital Innovation and Transformation

**Unlocking the Power of Data and Innovation**

At Elephant Ventures, we specialize in transforming your business through data-driven insights, AI solutions, and digital innovation. With over 20 years of proven success, we partner with leading enterprises and innovative startups to deliver exceptional results.

## Our Offerings

- **Data & Insights**: Rapidly capture value from data-driven decisions through workshops and visualization.
- **AI Services**: Explore the potential of your data with our free Generative AI proof of concept powered by AWS.
- **Digital Transformation**: Enable agile enterprises through capability-building and process optimization.
- **Innovation Services**: Accelerate your product journey with our rapid prototyping and MVP builds.
- **Data Engineering**: Modernize and streamline your data infrastructure for better integration and analytics.

## Why Choose Elephant?

Our clients — including Fortune 500 companies and startups — trust us for our exceptional technical expertise, problem-solving abilities, and proactive approach. Success stories include helping a multinational bank enhance digital initiatives and leading a Fortune 1000 client through a digital transformation rescue.

### Customer Testimonials

> *"Elephant's technical expertise and problem-solving abilities are exceptional. They're true partners."*  
> — Stefanie Youmans, Sr. Director, Program Management & Technology

> *"We had a clear MVP backlog and were sprinting and delivering value within weeks."*  
> — Kirk Voltz, SVP Strategic Partnerships, Financial Services

## Company Culture

At Elephant Ventures, we foster a culture of collaboration, innovation, and continuous learning. Our team embodies an ownership mindset, ensuring that every project is approached with dedication and care for our clients’ success. We regularly assess our work-life balance, recently piloting a 4-Day Work Week to enhance employee well-being and productivity.

## Careers at Elephant

We are always on the lookout for innovative thinkers and doers to join our diverse team. If you are passionate about technology and excited to contribute to groundbreaking projects across industries, explore our career opportunities and help shape the future of digital transformation.

### Join Us!

Are you ready to embark on a journey of transformation? **Contact us** or **book a call** today!

- **Email**: [info@elephantventures.com](mailto:info@elephantventures.com)
- **Phone**: 1-212-730-6710

**Unlock the future with Elephant Ventures!**
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [31]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("Elephant Ventures, LLC.", "https://elephantventures.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.elephantventures.com/data-to-decision/'}, {'type': 'services page', 'url': 'https://www.elephantventures.com/ai-services/'}, {'type': 'blog page', 'url': 'https://www.elephantventures.com/blog/'}]}


# Welcome to Elephant Ventures, LLC: The Mischievous Titans of Digital Transformation

### Are you tired of the mundane and uninspired? Come indulge in our brand of audacious innovation!

At Elephant Ventures, we take pride in shaking things up. For **20 years**, we've been the wickedly clever choice of both startups and Fortune 500 companies, leveraging the latest in AI, data engineering, and cloud technology. Whether you’re looking for transformative insights or simply want someone to make your tech woes disappear, we’re here to help — while basking in the glory of our genius, of course.

---

## Who We Serve: Your Dreams, Our Playground

Leading enterprises and ambitious startups alike choose us as their trusted partner. We bring your wild visions to life by throwing your data into our cauldron, concocting **Data-Driven Decisions** that would make any wizard jealous. 

### But don’t just take our word for it:
- **Stefanie Youmans**, Senior Director, describes us as “true partners.” (Hmm... is that a backhanded compliment?)
- **Kirk Voltz**, SVP Strategic Partnerships, praises our ability to charge full steam ahead — in mere weeks! 
- **Amelia Stephens**, VP/Director of US Retail Channel Marketing, seems to think we have an uncanny knack for asking just the right strategic questions. 

---

## The Secret Sauce: What We Do 

Shattered expectations from a previous failed attempt? We’ve got your back with our **Dependable Transformation** process! With our **Rapid Planning Workshops**, **Data Engineering**, and sexy **AI Services**, we'll sprinkle magic dust on your projects, delivering **MVPs** faster than you can say “what do you mean ‘budget constraints’?”

### Our Services Include:
- Generative AI and Disruptive Tech 🎩
- Digital Transformation from A to Z 💻
- Data Engineering that transcends human comprehension 📊
- Innovation Services so breathtaking, they should come with a fanfare 🎉

---

## Our Company Culture: The Mischief Makers

We’re not your run-of-the-mill corporate drones. Our secret ingredient is a sense of play. We’ve even tested a 4-day workweek—because who needs 40 hours when you can achieve maximum brilliance in four? Our **ownership mindset** means we don’t just coach; we roll up our sleeves and get our hands dirty right alongside you. It's messy, it’s delightful, and it’s effective.

---

## Career Opportunities: Join the Ranks of the Bold

Looking to join the ranks of the audacious? At Elephant Ventures, we’re always on the lookout for those with a penchant for disruption. Your mission, should you choose to accept it, will involve navigating the exhilarating waters of AI and digital transformation. If you want a career where you can challenge the status quo and hang out with brilliant innovators, throw your hat in our ring!

---

## Connect With Us - Embrace the Chaos

We promise a partnership filled with fantastic discoveries and cutting-edge tech explorations. Not interested in reaching out and shaping the future? Fair enough, more glory for us! 

- 📧 **Email:** [info@elephantventures.com](mailto:info@elephantventures.com)
- 📞 **Phone:** 1-212-730-6710

Make the bold choice today. Your competitors are trembling—we’d love for you to join the fun!

--- 

### Disclaimer:
Elephant Ventures is not responsible for any feelings of triumph or exhilaration. Side effects may include excessive confidence, groundbreaking ideas, and a parched thirst for more innovation.

--- 

**Follow us and add a sprinkle of chaos to your business transformation! 🚀**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>